#### Install required packages

In [ ]:
%pip install -r requirements.txt

Load all environment variables

In [ ]:
from dotenv import load_dotenv
from azure.identity.aio import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True)

chat_completion_model_name = os.getenv('AZURE_CHAT_COMPLETION_MODEL')

# The endpoint of APIM to reach the OpenAI instance
apim_openai_endpoint=os.getenv('APIM_OPENAI_ENDPOINT')

# We can or not use the API Key, here we will have two endpoint one with and one without
apim_subscription_key=os.getenv('APIM_SUBSCRIPTION_KEY')

# Default endpoint to get the token to call an Azure OpenAI instances
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

# Related to the REST api of Azure AI Foundry, change this value based on the one you configured
api_version = "2024-12-01-preview"

Create the AzureOpenAI Client

In [7]:
from openai import AsyncAzureOpenAI

client = AsyncAzureOpenAI(
    api_version=api_version,
    azure_endpoint=apim_openai_endpoint,
    api_key=apim_subscription_key # Here we pass the APIM key   
) 

Creating an helper method to call the model

In [8]:
async def prompt(prompt:str) -> str:
    response = await client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            }
        ],
        max_completion_tokens=2000,
        temperature=1.0,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0,
        model=chat_completion_model_name
    )  

    return response.choices[0].message.content     